In [1]:
import numpy as np
import re
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle 
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
def tokenize(text):
    """
    Return the list of non-unique tokenized words. The function transforms 
    all text to lowercase, removes any punctuation and filters any non-ascii characters
    if found. Lemmatize the word (Reducing the word to it's original root called lema
    for example the lema of changed, changing , changes is change), and drop words 
    length < 3
    """
    text = text.lower()
    #the following preprocessing code uses regular expression to match
    #any punctuation any digits 
    #carriage returns, tabs ,and new lines
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nonpunctText = regex.sub(" ",text)
    words = nonpunctText.split(' ')
    words = [word.encode('ascii','ignore').decode('ascii') for word in words]
    lmtzer = WordNetLemmatizer()
    words = [lmtzer.lemmatize(w) for w in words]
    words = [w for w in words if len(w) >2]
    return words

In [3]:
import pandas as pd
data = pd.read_csv("Toxic-Comment-Classification-Challenge-master/data/train.csv")
vector = TfidfVectorizer(ngram_range=(1, 1), analyzer='word',
                         tokenizer=tokenize, stop_words='english',
                         strip_accents='unicode',use_idf=True, min_df=10)
vector2 = vector.fit_transform(data["comment_text"])

/Users/ali_shar/opt/anaconda3/envs/AIprojectEnv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
labels = data.iloc[:,2:].columns

In [5]:
y_labels = data.iloc[:,2:]

In [6]:
#comments

NameError: name 'comments' is not defined

In [7]:
with open("project_variables/Xtrain.pickle",'rb') as file:
    comments = pickle.load(file)
Xtrain,XTest,Ytrain,YTest = train_test_split(comments,y_labels,test_size=0.2,random_state=42)

In [8]:
sample = vector.transform(["you are a dumb person"])

In [9]:
with open("project_variables/models/LogisticRegression.pickle",'rb') as file:
    model = pickle.load(file)

/Users/ali_shar/opt/anaconda3/envs/AIprojectEnv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.4.1.post1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
for label in labels:
    np.random.seed(42)
    model.fit(Xtrain,Ytrain[label])
    print(f"{label} {'True' if model.predict(sample) else 'False'}")

toxic True
severe_toxic False
obscene False
threat False
insult True
identity_hate False


In [ ]:
#!pip3 install --upgrade gradio jinja2

In [11]:
import gradio as gr

In [12]:
def score_comment(comment):
    vectorized_comment = vector.transform([comment])
    text = ''
    for label in labels:
        np.random.seed(42)
        model.fit(Xtrain,Ytrain[label])
        text+= '{}: {}   '.format(label,'True' if model.predict(vectorized_comment) else 'False')
    return text

In [13]:
interface = gr.Interface(fn=score_comment,
                        inputs=gr.components.Textbox(lines=2,placeholder='Comment to score'),
                        outputs='text')

In [ ]:
x = score_comment("i hate you. you are the worst person and dumb")
x

In [ ]:
x = 'test\ntest'.format()
print(x)

In [14]:
interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
interface.close()